In [3]:
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import html2text

In [37]:
print(f'You are a question-and-answer dataset generating expert, you are generating data that will be used to train a large language model designed to address questions posed by users regarding the game Minecraft, and from that, you will generate question-and-answer data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nYour task is to generate at least 20 examples.Make sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.')

You are a question-and-answer dataset generating expert, you are generating data that will be used to train a large language model designed to address questions posed by users regarding the game Minecraft, and from that, you will generate question-and-answer data samples, each with a prompt/response pair.

You will do so in this format:
```
prompt
-----------
$prompt_goes_here
-----------

response
-----------
$response_goes_here
-----------
```

Your task is to generate at least 20 examples.Make sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.


In [4]:
# URL of the page to scrape
# url = 'https://minecraft.fandom.com/wiki/Minecraft_Wiki'
url = 'https://minecraft.wiki/w/Diamond'
# Make a request to fetch the HTML content
response = requests.get(url)
html_content = response.text

In [5]:
with open('diamond.html', 'w') as f:
    f.write(html_content)

In [6]:
# Use Beautiful Soup to parse the HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Extract the main content of the page - adjust the selector based on your needs
main_content = soup.find('div', {'class': 'mw-parser-output'})

In [7]:
h1s = soup.find_all('h1')
print(h1s)
if len(h1s) > 1:
    print('unexpected h1s>1')
h1_text = None
for h1 in h1s:
    if h1['id'] == 'firstHeading':
        h1_text = h1.text
if not h1_text:
    print('firstheading not found')


[<h1 class="firstHeading mw-first-heading" id="firstHeading"><span class="mw-page-title-main">Diamond</span></h1>]


In [8]:
mw_parser_output = soup.find('div', {'class': 'mw-parser-output'})

In [9]:
first_p = mw_parser_output.find('p', recursive=False)
if not first_p:
    print('first_p not found')
else:
    first_p_text = first_p.get_text()
    print(first_p_text)

A diamond is a mineral that can be obtained from diamond ore, loot chests, and suspicious sand. It can be used to craft diamond tools and armor (and therefore obtain obsidian), and is required to craft enchanting tables, jukeboxes and duplicate smithing templates.



In [10]:
toc = mw_parser_output.find('div', {'class': 'toc'})
# print(toc)
ul = toc.find('ul', recursive=False)
lis = ul.find_all('li', recursive=False)

In [11]:
def convert_multilevel_toc(ul):
    sections = []
    lis = ul.find_all('li', recursive=False)
    for li in lis:
        section = {
            'title': li.a.get_text(),
        }
        ul_2 = li.find('ul')
        if ul_2:
            section['sub_sections'] = convert_multilevel_toc(ul_2)
        sections.append(section)
    return sections
toc = mw_parser_output.find('div', {'class': 'toc'})
if not toc:
    print('toc not found')
else:
    toc_title = toc.find('div', {'class': 'toctitle'})
    if not toc_title:
        print('toc_title not found')
    else:
        toc_title_text = toc_title.get_text()
        print(toc_title_text)
    toc_ul = toc.find('ul')
    if not toc_ul:
        print('toc_ul not found')
    else:
        toc_sections = convert_multilevel_toc(toc_ul)
        print(toc_sections)

Contents
[{'title': '1 Obtaining', 'sub_sections': [{'title': '1.1 Mining'}, {'title': '1.2 Chest loot'}, {'title': '1.3 Suspicious sand loot'}, {'title': '1.4 Crafting'}, {'title': '1.5 Smelting'}]}, {'title': '2 Usage', 'sub_sections': [{'title': '2.1 Crafting ingredient'}, {'title': '2.2 Trading'}, {'title': '2.3 Repairing'}, {'title': '2.4 Beacons'}, {'title': '2.5 Smithing ingredient'}]}, {'title': '3 Data values', 'sub_sections': [{'title': '3.1 ID'}]}, {'title': '4 Achievements'}, {'title': '5 Advancements'}, {'title': '6 History'}, {'title': '7 Issues'}, {'title': '8 Trivia'}, {'title': '9 Gallery', 'sub_sections': [{'title': '9.1 Screenshots'}, {'title': '9.2 In other media'}]}, {'title': '10 External links'}]


In [44]:
print(toc_sections)

[{'title': '1 Obtaining', 'sub_sections': [{'title': '1.1 Mining'}, {'title': '1.2 Chest loot'}, {'title': '1.3 Suspicious sand loot'}, {'title': '1.4 Crafting'}, {'title': '1.5 Smelting'}]}, {'title': '2 Usage', 'sub_sections': [{'title': '2.1 Crafting ingredient'}, {'title': '2.2 Trading'}, {'title': '2.3 Repairing'}, {'title': '2.4 Beacons'}, {'title': '2.5 Smithing ingredient'}]}, {'title': '3 Data values', 'sub_sections': [{'title': '3.1 ID'}]}, {'title': '4 Achievements'}, {'title': '5 Advancements'}, {'title': '6 History'}, {'title': '7 Issues'}, {'title': '8 Trivia'}, {'title': '9 Gallery', 'sub_sections': [{'title': '9.1 Screenshots'}, {'title': '9.2 In other media'}]}, {'title': '10 External links'}]


In [12]:
exclude_topic = ['Achievements', 'Advancements', 'History', 'Issues', 'Trivia', 'Gallery', 'References', 'External links']
def filter_section(section_title):
    if section_title in exclude_topic:
        return True
    return False

In [25]:

def get_table_content(table):
    rows = []
    column_count = 0
    trs = table.find_all('tr')
    row = []
    for th in trs[0].find_all('th'):
        row.append(th.get_text().strip())
        column_count += 1
    rows.append(row)
    for tr in trs[1:]:
        row = []
        tds = tr.find_all('td')
        ths = tr.find_all('th')
        for _ in range(column_count-len(tds)-len(ths)):
            row.append('')
        for th in ths:
            row.append(th.get_text().strip())
        for td in tds:
            # details differs, need to make sure all conditions are covered
            spans = td.find_all('span', recursive=False)
            row_text = ''
            if spans:
                for span in td.find_all('span'):
                    if span.get('class') is not None and 'sprite-text' in span['class']:
                        # the <pre> appears behind the <span> own text
                        # row.append(span.get_text().strip())
                        row_text += span.get_text().strip()
            if row_text:
                row.append(row_text)
            else:
                # row.append('')
                if td.contents:
                    content0 = td.contents[0]
                    if isinstance(content0, NavigableString):
                        row.append(content0.strip())
                    elif isinstance(content0, Tag):
                        row.append(td.get_text().strip())
                    else:
                        print('unexpected content0 type') 
                        row.append('')
                else:
                    row.append('')
        rows.append(row)
    return rows

In [14]:
# with open('test_data/table.txt', 'r') as f:
#     table_str = f.read()
# table = BeautifulSoup(table_str, 'html.parser')
# table_content = get_table_content(table)
# print(table_content[0])
# print(table_content[1])
# print(table_content[2])
# print(table_content[3])

In [32]:
h_names = [f'h{i}' for i in range(8)]
def get_all_content(content_area):
    all_content = []
    first_h2 = content_area.find('h2', recursive=False)
    # print(first_h2)
    h2_title = first_h2.find_all('span', recursive=False)[0].get_text()
    next_sib = first_h2
    skip_section = False
    if filter_section(h2_title):
        skip_section = True
        next_sib = next_sib.find_next_sibling()
        print(f'start skiping: {h2_title}')
    while next_sib:
        for i, h_name in enumerate(h_names):
            if next_sib.name == h_name:
                level = 2
                h_title = next_sib.find_all('span', recursive=False)[0].get_text()
                if i == 2:
                    if filter_section(h_title):
                        next_sib = next_sib.find_next_sibling()
                        skip_section = True
                        print(f'start skiping: {h_title}')
                        continue
                    else:
                        print(f'stop skiping: {h_title}')
                        skip_section = False
                else:
                    level = i
                if not skip_section:
                    all_content.append(
                        {
                            'type': 'title',
                            'level': level,
                            'content': h_title
                        }
                    )
                next_sib = next_sib.find_next_sibling()
                continue
        if skip_section:
            next_sib = next_sib.find_next_sibling()
            continue
        if next_sib.name == 'table':
            table_content = get_table_content(next_sib)
            all_content.append(
                {
                    'type': 'table',
                    'content': table_content
                }
            )
        else:
            # assume all other content is text
            all_content.append(
                {
                    'type': 'text',
                    'content': next_sib.get_text()
                }
            )
        next_sib = next_sib.find_next_sibling()
    return all_content

In [33]:
h2 = mw_parser_output.find('h2')
# print(first_h2.find_all('span')[0].get_text())
all_content = get_all_content(mw_parser_output)

stop skiping: Obtaining
stop skiping: Usage
stop skiping: Data values
start skiping: Achievements
start skiping: Advancements
start skiping: History
start skiping: Issues
start skiping: Trivia
start skiping: Gallery
start skiping: External links


In [17]:
for i in range(7):
    print(all_content[i])

{'type': 'title', 'level': 2, 'content': 'Obtaining'}
{'type': 'text', 'content': 'See also: Tutorials/Diamonds\n'}
{'type': 'title', 'level': 3, 'content': 'Mining'}
{'type': 'text', 'content': 'See also: Diamond Ore\xa0§\xa0Natural generation\n'}
{'type': 'text', 'content': "Diamond ore can be mined using an iron pickaxe or better. An ore drops a single diamond. If mined by any other tool, it drops nothing. If the pickaxe is enchanted with Fortune, it has a chance of dropping an extra diamond per level of Fortune, allowing for a maximum of 4 diamonds with Fortune III. If the ore is mined using a pickaxe enchanted with Silk Touch, it drops an ore block rather than a diamond. The ore is found at level 15 or lower, most commonly between -50 and -64; levels -58 and -59 are the preferred levels to mine at since they don't contain bedrock, as it starts spawning at -60.\n"}
{'type': 'title', 'level': 3, 'content': 'Chest loot'}
{'type': 'table', 'content': [['Item', 'Structure', 'Container'

In [18]:
table = all_content[6]['content']
for i in range(6):
    print(table[i])

['Item', 'Structure', 'Container', 'Quantity', 'Chance']
['', '', '', '', 'Java Edition']
['Diamond', 'Mineshaft', 'Chest', '1–2', '8.9%']
['', 'Bastion Remnant', 'Treasure\xa0chest', '2–6', '14.3%']
['', 'Buried Treasure', 'Chest', '1–2', '59.9%']
['', 'Desert Pyramid', 'Chest', '1–3', '6.3%']


In [19]:
def to_md_table(table_content: list[list[str]]):
    if not table_content:
        return ""

    # Determine maximum width of each column
    column_widths = [max(len(str(item)) for item in column) for column in zip(*table_content)]

    # Create the Markdown table string
    markdown_table = ""

    # Add header row
    try:
        header_row = table_content[0]
        markdown_table += "| " + " | ".join(f"{item:{column_widths[i]}}" for i, item in enumerate(header_row)) + " |\n"
    except Exception as e:
        print(column_widths)
        print(table_content)
        print(header_row)

    # Add separator row
    markdown_table += "|-" + "-|-".join("-" * width for width in column_widths) + "-|\n"

    # Add data rows
    for row in table_content[1:]:
        try:
            markdown_table += "| " + " | ".join(f"{item:{column_widths[i]}}" for i, item in enumerate(row)) + " |\n"
        except Exception as e:
            print(column_widths)
            print(row)

    return markdown_table+'\n'

def to_header(title: str, level: int):
    return f"{'#' * level} {title}\n"


In [30]:
def write_all_content(file_name, all_content):
    with open(file_name, 'a') as f:
        for content in all_content:
            if content['type'] == 'title':
                f.write(to_header(content['content'], content['level']))
            elif content['type'] == 'table':
                f.write(to_md_table(content['content']))
            elif content['type'] == 'text':
                f.write(content['content']+'\n')

file_name = 'diamond.md'

write_all_content(file_name, all_content)



In [45]:
'https://minecraft.wiki/w/Diamond_Hoe'.split('/')

['https:', '', 'minecraft.wiki', 'w', 'Diamond_Hoe']